In [ ]:
from pathlib import Path
from typing import List, Union
import pandas as pd
import re

def _detect_frame_index_pose(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if 'scorer' in df.columns and pd.api.types.is_integer_dtype(df['scorer']):
        return df.set_index('scorer', drop=True).rename_axis('frame')
    for col in ('frame', 'index', 'Unnamed: 0'):
        if col in df.columns and pd.api.types.is_integer_dtype(df[col]):
            return df.set_index(col, drop=True).rename_axis('frame')
    return df.reset_index(drop=True).rename_axis('frame')

def _detect_frame_index_bbox(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if 'frame' in df.columns:
        return df.set_index('frame', drop=True)
    if 'Unnamed: 0' in df.columns:
        return df.rename(columns={'Unnamed: 0': 'frame'}).set_index('frame', drop=True)
    if not isinstance(df.index, pd.RangeIndex):
        return df.reset_index(drop=False).rename(columns={'index': 'frame'}).set_index('frame', drop=True)
    return df.rename_axis('frame')

def _xy_like_multiindex_cols(columns: pd.MultiIndex):
    x_cols, y_cols = [], []
    for col in columns:
        last = col[-1]
        if isinstance(last, str):
            if last.startswith('x') and 'var' not in last: x_cols.append(col)
            if last.startswith('y') and 'var' not in last: y_cols.append(col)
    return x_cols, y_cols

def _xy_like_flat_cols(columns: pd.Index):
    x_cols, y_cols = [], []
    for c in columns.astype(str):
        if 'var' in c.lower(): 
            continue
        if c == 'x' or c.endswith('_x') or c.startswith('x_'): x_cols.append(c)
        if c == 'y' or c.endswith('_y') or c.startswith('y_'): y_cols.append(c)
    return x_cols, y_cols

def translate_pose_by_bbox(pose_df: pd.DataFrame, bbox_df: pd.DataFrame, mode: str = "subtract") -> pd.DataFrame:
    """Map full-frame coords -> bbox-cropped coords (mode='subtract').
       Use mode='add' to go back to full-frame."""
    pose_df = _detect_frame_index_pose(pose_df)
    bbox_df = _detect_frame_index_bbox(bbox_df)
    common = pose_df.index.intersection(bbox_df.index)
    pose_df, bbox_df = pose_df.loc[common].copy(), bbox_df.loc[common].copy()
    if not {'x','y'}.issubset(bbox_df.columns):
        raise ValueError(f"bbox_df must have 'x' and 'y'; got {bbox_df.columns}")
    sign = -1 if mode == "subtract" else 1

    if isinstance(pose_df.columns, pd.MultiIndex):
        x_cols, y_cols = _xy_like_multiindex_cols(pose_df.columns)
    else:
        x_cols, y_cols = _xy_like_flat_cols(pose_df.columns)

    if x_cols: pose_df.loc[:, x_cols] = pose_df.loc[:, x_cols].add(sign * bbox_df['x'], axis=0)
    if y_cols: pose_df.loc[:, y_cols] = pose_df.loc[:, y_cols].add(sign * bbox_df['y'], axis=0)
    return pose_df

def batch_translate_pose_csvs(
    pose_csvs: List[Union[str, Path]],
    bbox_csvs: List[Union[str, Path]],
    output_dir: Union[str, Path],
    mode: str = "subtract",
    suffix: str = "",
):
    if len(pose_csvs) != len(bbox_csvs):
        raise ValueError("pose_csvs and bbox_csvs must be same length (one per view).")
    output_dir = Path(output_dir); output_dir.mkdir(parents=True, exist_ok=True)
    outs = []
    for pose_csv, bbox_csv in zip(pose_csvs, bbox_csvs):
        # Try MultiIndex header first (Lightning Pose/EKS), else flat
        try: pose_df = pd.read_csv(pose_csv, header=[0,1,2])
        except Exception: pose_df = pd.read_csv(pose_csv)
        bbox_df = pd.read_csv(bbox_csv)
        translated = translate_pose_by_bbox(pose_df, bbox_df, mode=mode)
        out_path = output_dir / f"{Path(pose_csv).stem}.csv"
        translated.to_csv(out_path)
        outs.append(out_path)
    return outs

In [7]:
pose_csvs = ["./outputs/chickadee-preds/video_preds/PRL43_200617_131904_lBack.short.csv"]          # one per view
bbox_csvs  = ["./data/bounding_boxes/PRL43_200617_131904_lBack.short_bbox.csv"]  # matching order
out_files = batch_translate_pose_csvs(pose_csvs, bbox_csvs, output_dir="./outputs/cropped_csvs")
print(out_files)

[WindowsPath('outputs/cropped_csvs/PRL43_200617_131904_lBack.short_cropped.csv')]


In [ ]:
import cv2

path = "./videos/chickadee/PRL43_200617_131904_lBack.short.mp4"  # update to your file path
cap = cv2.VideoCapture(path)
if not cap.isOpened():
    raise RuntimeError("Could not open video")

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration_sec = frame_count / fps if fps > 0 else None

cap.release()

print(f"Frames: {frame_count}")
print(f"FPS: {fps}")
print(f"Duration (s): {duration_sec:.3f}" if duration_sec else "Duration unknown")

Frames: 1800
FPS: 60.0
Duration (s): 30.000
